In [2]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [19]:
from cmf.clean import cleaning_function
from cmf.clean import steps
from cmf.data import utils as du
from cmf.data import DB, Clusters, Table

from dotenv import load_dotenv, find_dotenv
import os

load_dotenv(find_dotenv())

df = du.generate_dummy_df()

df

,irrational,rounded
0,pi,3.14
1,e,2.72
2,phi,1.62


In [9]:
def to_upper(column: str) -> str:
    """
    All characters to uppercase
    """
    return f"upper({column})"

logger = logging.getLogger(__name__)

logger.info(
    "Creating probabilities table "
    "test"
)

In [31]:
clusters._db_expected_fields

['uuid', 'id', 'cluster', 'source', 'n']

In [7]:
x = set(["id", "uuid", "name"])
y = set(["id", "uuid", "name"])
z = set(["id"])
a = set(["id", "foo"])

y <= x
z <= x
a <= x

True

True

False

In [5]:
x = {"foo": "bar", "baz": "quz"}
y = {"foo": "bar"}
len(x)
len(y)
for i, j in x.items():
    print(i)
    print(j)

2

1

foo
bar
baz
quz


In [8]:
from pydantic import BaseModel

class Test(BaseModel):
    x: str
    y: str

    @classmethod
    def make_test(cls, foo: str) -> "Test":
        return cls(
            x=foo[0],
            y=foo[-1]
        )

test = Test.make_test(foo="lorem")
test.x
test.y

'l'

'm'

In [15]:
p, q = tuple(y.items())[0]
p
q

'foo'

'bar'

In [19]:
from cmf import query, process
from cmf.helpers import selector, cleaner, cleaners
from cmf.clean import company_name, company_number

select_ch = selector(
    table="companieshouse.companies", fields=["company_number", "company_name"]
)
ch_sample = query(select=select_ch, sample=0.05)

cleaner_name = cleaner(function=company_name, arguments={"column": "company_name"})
cleaner_number = cleaner(
    function=company_number, arguments={"column": "company_number"}
)
cleaner_name_number = cleaners(cleaner_name, cleaner_number)

ch_sample_cleaned = process(data=ch_sample, pipeline=cleaner_name_number)

In [20]:
ch_sample_cleaned

,company_number,company_name
0,14853598,1st prize competitions
1,7226001,1st procurement
2,14156025,1st production
3,7426911,1st projects
4,5043585,1st properties
...,...,...
2264,10868214,zen accounting solutions
2265,501712,zen accounts
2266,14856432,zen acer
2267,11804555,zen activewear
